In [1]:
#import os
#from datetime import datetime
#from datetime import date
import pymysql
from sqlalchemy import create_engine
from decouple import config 
from dotenv import load_dotenv
import matplotlib.pyplot as plt
import seaborn as sns
#from enum import Enum
from pandas import read_excel, read_sql_query

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
#pd.set_option("max_columns",None)


In [2]:
load_dotenv()
# get the environment variables needed
USER= config('USRCaris')
PASSWORD= config('PASSCaris')
HOSTNAME= config('HOSTCaris')
DBNAME= config('DBCaris')

# get the engine to connect and fetch
engine = create_engine(f"mysql+pymysql://{USER}:{PASSWORD}@{HOSTNAME}/{DBNAME}")
query = f'''
    SELECT 
    (a.score_7 + a.score_8a + score_9a + score_9b + score_17a + score_21 + score_21a + score_22 + score_22a + 1) AS parenting_score_total,
    p.patient_code AS dreams_code,
    dga.nbr_presence,
    a.*
FROM
    (SELECT 
        CASE
                WHEN
                    d.a7_ak_kils_w_ap_viv_10_14 = '0'
                        OR d.a7_Ak_kiles_w_ap_viv_15_19 = '0'
                THEN
                    2
                WHEN
                    d.a7_ak_kils_w_ap_viv_10_14 = '1'
                        OR d.a7_Ak_kiles_w_ap_viv_15_19 = '1'
                THEN
                    4
                WHEN
                    d.a7_ak_kils_w_ap_viv_10_14 = '001'
                        OR d.a7_Ak_kiles_w_ap_viv_15_19 = '01'
                THEN
                    4
                WHEN
                    d.a7_ak_kils_w_ap_viv_10_14 = '03'
                        OR d.a7_Ak_kiles_w_ap_viv_15_19 = '2'
                THEN
                    4
                WHEN
                    d.a7_ak_kils_w_ap_viv_10_14 = 'lt_presize'
                        OR d.a7_Ak_kiles_w_ap_viv_15_19 = 'lt_presize'
                THEN
                    2
            END AS score_7,
            IF(d.a8_eske_gen_yon_moun_ou_santi_ou_ka_al_kote_ = '0', 0, IF(d.a8_eske_gen_yon_moun_ou_santi_ou_ka_al_kote_ = '2', 4, 0)) AS score_8a,
            IF(d.a9_ou_gen_lot_kote_ou_pa_santi_ou_ansekirite = '1', 2, 0) AS score_9a,
            IF(d.b9_list_kote_ou_pa_an_sekirite IN ('1' , '3', '2', '04'), 4, IF(d.b9_list_kote_ou_pa_an_sekirite = '4', 8, IF(d.b9_list_kote_ou_pa_an_sekirite = '0001', 2, 0))) AS score_9b,
            IF(a17 IN ('4' , '04', '004'), 8, 0) AS score_17a,
            IF(d.eske_ou_konn_bwe_alkol = 'oui', 3, 0) AS score_21,
            IF(d.a21_ske_ou_bw_alkl_osnon_itilize_lt_dwg_10_14 = '3'
                OR a21_nan_ki_frenkans_ou_itilize_alkol = '2', 6, IF(d.a21_ske_ou_bw_alkl_osnon_itilize_lt_dwg_10_14 = '5'
                OR a21_nan_ki_frenkans_ou_itilize_alkol = '5', 8, 0)) AS score_21a,
            IF(d.eske_ou_konn_itilize_dwog = 'oui', 3, 0) AS score_22,
            IF(d.ske_ou_konn_itilize_dwg_10_14 = '3'
                OR ske_ou_konn_itilize_dwg_15_19 = '2', 6, IF(d.ske_ou_konn_itilize_dwg_10_14 = '5'
                OR ske_ou_konn_itilize_dwg_15_19 = '5', 8, 0)) AS score_22a,
            d.*
    FROM
        caris_db.dreams_surveys_data d
    WHERE
        (TIMESTAMPDIFF(YEAR, d.nan_ki_dat_ou_fet, NOW()) BETWEEN 10 AND 17)
            AND (d.a7_ak_kils_w_ap_viv_10_14 NOT IN ('8' , '6')
            OR d.a7_ak_kils_w_ap_viv_10_14 IS NULL)
            AND (d.a7_Ak_kiles_w_ap_viv_15_19 NOT IN ('5' , '3')
            OR d.a7_Ak_kiles_w_ap_viv_15_19 IS NULL)) a
        LEFT JOIN
    dream_member dm ON dm.case_id = a.case_id
        LEFT JOIN
    patient p ON p.id = dm.id_patient
        LEFT JOIN
    (SELECT 
        count(distinct dgs.topic) AS nbr_presence, id_patient
    FROM
        dream_group_attendance dga1
        left join dream_group_session dgs on dga1.id_group_session=dgs.id where dga1.value="P"
    GROUP BY dga1.id_patient) dga ON dga.id_patient = dm.id_patient
WHERE
    a.total > 14
ORDER BY (a.score_7 + a.score_8a + score_9a + score_9b + score_17a + score_21 + score_21a + score_22 + score_22a + 1) DESC
'''

parenting = read_sql_query(query,engine,parse_dates=True)
# get the test excel file from Query

# close the pool of connection
engine.dispose()

True

In [3]:
parenting.dreams_code.fillna("NULL",inplace=True)
parenting.dreams_code.count()

29693

In [4]:
parenting.to_excel('all_parenting_elligible.xlsx',na_rep="", index=False)

In [5]:
parenting.columns

Index(['parenting_score_total', 'dreams_code', 'nbr_presence', 'score_7',
       'score_8a', 'score_9a', 'score_9b', 'score_17a', 'score_21',
       'score_21a',
       ...
       'a20_swiv_tretman_pou_lot_ist',
       'ou_konn_enkonsyan_apre_alcohol_ou_dwog', 'ou_konnen_sant_pou_planin',
       'ou_konnen_kote_pou_sevis_sante_tretman_mal',
       'tifi_a_nan_nivo_ti_ris_ou_wel_elijib', 'timeEnd', 'timeStart',
       'agent_username', 'created_ad', 'organisation'],
      dtype='object', length=213)

# Recherche

In [6]:
selected = read_excel('./data/parent_update.xlsx')

In [7]:
selected.case_id.count()

2593

In [8]:
# get the engine to connect and fetch
engine = create_engine(f"mysql+pymysql://{USER}:{PASSWORD}@{HOSTNAME}/{DBNAME}")
query = '''
SELECT 
    d.case_id,
    dm.id_patient as id_patient,
    p.patient_code AS code,
    d.a_non_patisipan_an AS first_name,
    d.b_siyati AS last_name,
    TIMESTAMPDIFF(YEAR,
        d.nan_ki_dat_ou_fet,
        now()) AS age,
    d.nan_ki_dat_ou_fet AS dob,
    d.a1_dat_entvyou_a_ft_jjmmaa_egz_010817 AS interview_date,
    d.e__telefn,
    d.d_adrs AS adress,
    IF(dm.id IS NOT NULL, 'yes', 'no') AS already_in_a_group,
    dm.id_group AS actual_id_group,
    dg.name AS actual_group_name,
    dm.id_parenting_group AS actual_id_parenting_group,
    dpg.name AS actual_parenting_group_name,
    dh.name AS actual_hub,
    ld.name AS actual_departement,
    d.f_komin AS commune,
    d.g_seksyon_kominal AS commune_section,
    d.b1_non_moun_mennen_entvyou_a AS interviewer_firstname,
    d.c1_siyati_moun_ki_f_entvyou_a AS interviewer_lastname,
    d.d1_kad AS interviewer_role,
    d.lot_kad AS interviewer_other_info,
    d.h_kote_entvyou_a_ft AS interview_location,
    d.paran_ou_vivan AS is_your_parent_alive,
    d.i_non_manman AS mothers_name,
    d.j_non_papa AS fathers_name,
    d.k_reskonsab_devan_lalwa AS who_is_your_law_parent,
    d.total,
    d.organisation
FROM
    caris_db.dreams_surveys_data d
        LEFT JOIN
    dream_member dm ON dm.case_id = d.case_id
        LEFT JOIN
    patient p ON p.id = dm.id_patient
        LEFT JOIN
    dream_group dg ON dg.id = dm.id_group
        LEFT JOIN
    dream_group dpg ON dpg.id = dm.id_parenting_group
        LEFT JOIN
    dream_hub dh ON dh.id = dg.id_dream_hub
        LEFT JOIN
    lookup_commune lc ON lc.id = dh.commune
        LEFT JOIN
    lookup_departement ld ON ld.id = lc.departement
'''

sdata = read_sql_query(query,engine,parse_dates=True)
# get the test excel file from Query

# close the pool of connection
engine.dispose()

In [10]:
update_parenting =  sdata[sdata.case_id.isin(selected.case_id)]
update_parenting.case_id.count()

2593

In [11]:
update_parenting.to_excel('update_parenting.xlsx',index=False,na_rep="")